In [70]:
import glob
import json
import os
import random
import re
import shutil
import copy
from collections import defaultdict

import pandas as pd
import pdfplumber
import seaborn as sns

from sklearn.model_selection import train_test_split

import utils.preprocessing_portuguese as preprossPT

In [71]:
df_input = pd.read_csv('/dados01/workspace/ufmg.f01dcc/m03/business_understanding/notebooks/lstm_data/preprocessed_data_v2.csv')
display(df_input.head())

,doc_id,final_meta-class,label,label_int,city,file_dir,one_page,two_pages,three_pages,four_pages,fold,four_pages_processed
0,499fa518c7724a6e449c509b7d7bc819,ATA,ata dispensa licitacao,0,cristais,../data/290-licitacoes-cristais/data/files_jso...,comissao permanente licitacoes ata dispensa ju...,comissao permanente licitacoes ata dispensa ju...,comissao permanente licitacoes ata dispensa ju...,comissao permanente licitacoes ata dispensa ju...,val,comissao permanente licitacoes ata dispensa ju...
1,56a09c5d1d04cc95ada4d68ad22dcbd9,ATA,ata dispensa licitacao,0,cristais,../data/290-licitacoes-cristais/data/files_jso...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,test,comissao permanente licitacoes dia fevereiro r...
2,6e9f35208290c1130fcccad47b50aa53,ATA,ata dispensa licitacao,0,cristais,../data/290-licitacoes-cristais/data/files_jso...,praca cel centro estado cep pabx cnpj comissa...,praca cel centro estado cep pabx cnpj comissa...,praca cel centro estado cep pabx cnpj comissa...,praca cel centro estado cep pabx cnpj comissa...,val,praca cel centro estado cep pabx cnpj comissao...
3,669c029c5812ec9a31b9c211044157b4,ATA,ata dispensa licitacao,0,cristais,../data/290-licitacoes-cristais/data/files_jso...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,comissao permanente licitacoes dia fevereiro r...,val,comissao permanente licitacoes dia fevereiro r...
4,a22421dc45d623c9200c7e8fb1e6ca34,ATA,ata dispensa licitacao,0,cristais,../data/290-licitacoes-cristais/data/files_jso...,comissao permanente licitacoes maio reuniu com...,comissao permanente licitacoes maio reuniu com...,comissao permanente licitacoes maio reuniu com...,comissao permanente licitacoes maio reuniu com...,train,comissao permanente licitacoes maio reuniu com...


In [72]:
def split_stratified_into_train_val_test(df_input, stratify_colname='final_meta-class',
                                         frac_train=0.7, frac_val=0.2, frac_test=0.1,
                                         random_state=42):
    '''
    Source: https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
    Splits a Pandas dataframe into three subsets (train, val, and test)
    following fractional ratios provided by the user, where each subset is
    stratified by the values in a specific column (that is, each subset has
    the same relative frequency of the values in the column). It performs this
    splitting by running train_test_split() twice.

    Parameters
    ----------
    df_input : Pandas dataframe
        Input dataframe to be split.
    stratify_colname : str
        The name of the column that will be used for stratification. Usually
        this column would be for the label.
    frac_train : float
    frac_val   : float
    frac_test  : float
        The ratios with which the dataframe will be split into train, val, and
        test data. The values should be expressed as float fractions and should
        sum to 1.0.
    random_state : int, None, or RandomStateInstance
        Value to be passed to train_test_split().

    Returns
    -------
    df_train, df_val, df_test :
        Dataframes containing the three splits.
    '''
    print(round(frac_train + frac_val + frac_test))
    if round(frac_train + frac_val + frac_test) != 1:
        raise ValueError('fractions %f, %f, %f do not add up to 1.0' %  (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s is not a column in the dataframe' % (stratify_colname))

    X = df_input # Contains all columns.
    y = df_input[[stratify_colname]] # Dataframe of just the column on which to stratify.

    # Split original dataframe into train and temp dataframes.
    df_train, df_temp, y_train, y_temp = train_test_split(X,
                                                          y,
                                                          stratify=y,
                                                          test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Split the temp dataframe into val and test dataframes.
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp,
                                                      y_temp,
                                                      stratify=y_temp,
                                                      test_size=relative_frac_test,
                                                      random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test)

    return df_train, df_val, df_test, y_train, y_val, y_test

In [73]:
df_train, df_val, df_test, y_train, y_val, y_test = split_stratified_into_train_val_test(df_input, stratify_colname='final_meta-class',
                                         frac_train=0.7, frac_val=0.2, frac_test=0.1,
                                         random_state=42)

1


In [74]:
df_train

,doc_id,final_meta-class,label,label_int,city,file_dir,one_page,two_pages,three_pages,four_pages,fold,four_pages_processed
4496,cad98d60b59929099e8dcd304518523a,OUTROS,outros,8,cristais,../data/290-licitacoes-cristais/data/files_jso...,praca cel centro pabx cep estado cumpridas fo...,praca cel centro pabx cep estado cumpridas fo...,praca cel centro pabx cep estado cumpridas fo...,praca cel centro pabx cep estado cumpridas fo...,train,praca cel centro pabx cep estado cumpridas for...
3659,70e142c5b1964cfbf203a78dfa8ef217,OUTROS,aviso,9,ijaci,../data/353-licitacoes-ijaci/data/files_json/7...,estado mg aviso resultado julgamento homologa...,estado mg aviso resultado julgamento homologa...,estado mg aviso resultado julgamento homologa...,estado mg aviso resultado julgamento homologa...,train,estado mg aviso resultado julgamento homologac...
4182,028b47986f9c51ad6b669ab7d581a395,OUTROS,contrato,10,coqueiral,../data/289-licitacoes-coqueiral/data/files_js...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,train,contrato administrativo processo administrativ...
4400,f18c65bb7a36857b55db88ab8e1fe566,OUTROS,contrato,10,itamarati,../data/351-licitacoes-itamarati/licitacoes_it...,minas estado cnpj inscricao estadual isento c...,minas estado cnpj inscricao estadual isento c...,minas estado cnpj inscricao estadual isento c...,minas estado cnpj inscricao estadual isento c...,train,minas estado cnpj inscricao estadual isento co...
554,8336f956d75b5e87b7c58284b4a9fbc6,ATA,ata registro precos,3,coqueiral,../data/289-licitacoes-coqueiral/data/files_js...,ata registro precos processo administrativo l...,ata registro precos processo administrativo l...,ata registro precos processo administrativo l...,ata registro precos processo administrativo l...,train,ata registro precos processo administrativo li...
...,...,...,...,...,...,...,...,...,...,...,...,...
4298,fe14f995e918a321d86c2e47b4defa53,OUTROS,contrato,10,coqueiral,../data/289-licitacoes-coqueiral/data/files_js...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,train,contrato administrativo processo administrativ...
3941,240e488718bf5222ecea77f3446f9ddd,OUTROS,contrato,10,coqueiral,../data/289-licitacoes-coqueiral/data/files_js...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,contrato administrativo processo administrati...,val,contrato administrativo processo administrativ...
2749,35a36eb8a4ecc9063610873a78581b07,EDITAL,edital,4,ijaci,../data/353-licitacoes-ijaci/data/files_json/3...,estado edital licitacao processo licitatorio ...,estado edital licitacao processo licitatorio ...,estado edital licitacao processo licitatorio ...,estado edital licitacao processo licitatorio ...,train,estado edital licitacao processo licitatorio p...
2003,3b00559a2a9f13a697463bacc58a3ce1,EDITAL,edital,4,coqueiral,../data/289-licitacoes-coqueiral/data/files_js...,processo administrativo licitatorio modalidad...,processo administrativo licitatorio modalidad...,processo administrativo licitatorio modalidad...,processo administrativo licitatorio modalidad...,train,processo administrativo licitatorio modalidade...


In [55]:
text_preprocessing = preprossPT.TextPreProcessing()

ATA, HOMOLOG, EDITAL, OUTROS = "ATA", "HOMOLOG", "EDITAL", "OUTROS"

title_keys = ["ata_title_count", "homolog_title_count", "edital_title_count", "outros_title_count"]
content_keys = ["ata_content_count", "homolog_content_count", "edital_content_count","outros_content_count"]
folder_paths = [
    "../data/288_licitacoes-pirapetinga/data/files/*.pdf",
    "../data/289-licitacoes-coqueiral/data/files/*.pdf",
    "../data/290-licitacoes-cristais/data/files/*.pdf",
    "../data/302-licitacoes-olaria/data/files/*.pdf",
    "../data/304-licitacoes-passa-vinte/data/files/*.pdf",
    "../data/306-licitacoes-arantina/data/files/*.pdf",
    "../data/353-licitacoes-ijaci/data/files/*.pdf",
    "../data/381-licitacoes-sao-bento-abade/data/files/*.pdf",
    "../data/48-licitacoes-bh/data/files/*.pdf",
]

keywords = [
    {
        "word": "ata",
        "title_regex": str(r"\bata\b"),
        "content_regex": str(r"\bata\b"),
        "class": ATA,
    },
    {
        "word": "sessão pública",
        "title_regex": str(r"\bsessão pública\b"),
        "content_regex": str(r"\bsessão pública\b"),
        "class": ATA,
    },
    {
        "word": "homolog",
        "title_regex": str(r"\bhomologação\b"),
        "content_regex": str(r"\bhomologação\b"),
        "class": HOMOLOG,
    },
    {
        "word": "adjudicação",
        "title_regex": str(r"\badjudicação\b"),
        "content_regex": str(r"\badjudicação\b"),
        "class": HOMOLOG,
    },
    {
        "word": "convite",
        "title_regex": str(r"\bconvite\b"),
        "content_regex": str(r"\bconvite\b"),
        "class": EDITAL,
    },
    {
        "word": "edital",
        "title_regex": str(r"\bedital\b"),
        "content_regex": str(r"\bedital\b"),
        "class": EDITAL,
    },
    {
        "word": "cronograma",
        "title_regex": str(r"\bcronograma\b"),
        "content_regex": str(r"\bcronograma\b"),
        "class": OUTROS,
    },
    {
        "word": "aditamento",
        "title_regex": str(r"\baditamento\b"),
        "content_regex": str(r"\baditamento\b"),
        "class": OUTROS,
    },
    {
        "word": "retificação",
        "title_regex": str(r"\bretificação\b"),
        "content_regex": str(r"\bretificação\b"),
        "class": OUTROS,
    },
    {
        "word": "contrato administrativo",
        "title_regex": str(r"\bcontrato administrativo\b"),
        "content_regex": str(r"\bcontrato administrativo\b"),
        "class": OUTROS,
    },
    {
        "word": "ordem de serviço",
        "title_regex": str(r"\bordem de serviço\b"),
        "content_regex": str(r"\bordem de serviço\b"),
        "class": OUTROS,
    },
    {
        "word": "resposta",
        "title_regex": str(r"\bresposta\b"),
        "content_regex": str(r"\bresposta\b"),
        "class": OUTROS,
    },
    {
        "word": "extrato",
        "title_regex": str(r"\bextrato\b"),
        "content_regex": str(r"\bextrato\b"),
        "class": OUTROS,
    },
    {
        "word": "diário oficial",
        "title_regex": str(r"\bdiário oficial\b"),
        "content_regex": str(r"\bdiário oficial\b"),
        "class": OUTROS,
    },
    {
        "word": "aviso de",
        "title_regex": str(r"\baviso de\b"),
        "content_regex": str(r"\baviso de\b"),
        "class": OUTROS,
    },
]

pdfs = []

In [56]:
classes_of_interest = [ATA, HOMOLOG, EDITAL, OUTROS]
keywords_of_interest = [key_word for key_word in keywords if key_word["class"] in classes_of_interest]
limit = 400
minimum = 20

In [57]:
def get_pages(pdf):
    return pdf.pages

def title_extraction_bold(pdf_page, words_limit=20):
    title = ""
    for word in pdf_page.extract_words(extra_attrs=["fontname"])[:words_limit]:  # Using [2:] to skip the main title
        if "Bold" in word["fontname"]:
            title = " ".join([title, word["text"]]).strip()
    return title

def title_extraction_upper(pdf_page, words_limit=20):
    title = ""
    for word in pdf_page.extract_words(extra_attrs=["fontname"])[:words_limit]: 
        if word["text"].isupper():
            title = " ".join([title, word["text"]]).strip()
    return title

def title_extraction_upper_bold(pdf_page, words_limit=20):
    title = ""
    for word in pdf_page.extract_words(extra_attrs=["fontname"])[:words_limit]: 
        if "Bold" in word["fontname"] and word["text"].isupper():
            title = " ".join([title, word["text"]]).strip()
    return title

def title_extraction_breaklines(pdf, words_limit=20):
    page = pdf.pages[0]
    content = page.extract_text()
    first_lines = []
    if bool(content):
        content = text_preprocessing.remove_special_characters(content, exceptions=["\n"]).lower()
        content = text_preprocessing.remove_excessive_spaces(content)
        content = re.sub(r"\n\s*\n", "\n", content)
        first_lines = content.split("\n", 6)[:-1]
    else:
        return None
    return first_lines

def get_first_tokens(pdf, limit = 1000):
    num_pages = min(len(pdf.pages), 5)
    pages = pdf.pages[:num_pages]
    content = ''
    for page in pages:
        content += page.extract_text()[:limit]
    if bool(content):
        content = text_preprocessing.remove_special_characters(content).lower()
        content = text_preprocessing.remove_excessive_spaces(content)
    else:
        return None
    return content

def table_extraction(pdf, page_number = 0):
    tables = []
    page = pdf.pages[page_number]
    for table in page.extract_tables():
        tables.append(table)
    return tables

In [58]:
keywords = ['ata', 'sessão pública', 'homolog', 'adjudicação', 'convite', 'edital', 'cronograma', 'aditamento', 'retificação', 'contrato administrativo', 'ordem de serviço', 'resposta', 'extrato', 'diário oficial', 'aviso de']

# print(re.findall(r"(?=("+'|'.join(string_lst)+r"))", x))

[]


In [ ]:
def extract_title(title):
    return ' '.join(re.findall(r"(?=("+'|'.join(keywords)+r"))", title))

def extract_content(content):
    return ' '.join(re.findall(r"(?=("+'|'.join(keywords)+r"))", content))

cities_data = defaultdict(list)
empty = 0

for city in ["pirapetinga", "coqueiral", "cristais", "olaria", "passa vinte", "arantina", "ijaci", "sao bento abade"]:
    city_pdfs = [info for info in pdfs if info["city"] == city]
    
    pdf_i = 0
    print('-'*80)
    print(f'## {city} ##')
    
    while pdf_i < len(city_pdfs):
        try:
            pdf = city_pdfs[pdf_i]
            pdf_file = pdfplumber.open(pdf['path'])
            pdf_id = pdf["id"].split(".")[0]
        
            pages = get_pages(pdf_file)
            
            content = get_first_tokens(pdf_file)
            content = extract_content(pdf_file)
            
            title = title_extraction_breaklines(pdf_file)
            title = extract_title(pdf_file)
            
            
        except Exception as e:
            print("*-*-*", e)
        pdf_i += 1

In [ ]:
vocabulary = set()

default_features_dict = {'contrato_administrativo_t': 0, 'contrato_administrativo_c': 0, 'retificacao_t': 0, 'retificacao_c': 0, 
            'aviso_de_t': 0, 'aviso_de_c': 0, 'aditamento_t': 0, 'aditamento_c': 0, 'extrato_t': 0, 'extrato_c': 0, 
            'ordem_de_servico_t': 0, 'ordem_de_servico_c': 0, 'sessao_publica_t': 0, 'sessao_publica_c': 0, 
            'adjudicacao_t': 0, 'adjudicacao_c': 0, 'ata_t': 0, 'ata_c': 0, 'homologacao_t': 0, 'homologacao_c': 0, 
            'convite_t': 0, 'convite_c': 0, 'resposta_t': 0, 'resposta_c': 0, 'edital_t': 0, 'edital_c': 0, 
            'cronograma_t': 0, 'cronograma_c': 0, 'diario_oficial_t': 0, 'diario_oficial_c': 0}

def get_content_matches(pdf_id, title, content):
    features_dict = {'doc_id': pdf_id, 'title': title, 'city': city} 
    features_dict.update(copy.copy(default_features_dict))
    
    for word_dict in keywords:
        word = word_dict["word"]
        title_regex = word_dict["title_regex"]
        content_regex = word_dict["title_regex"]
        doc_class = word_dict["class"].lower()
        title_matches = []
        
        for index in range(len(title)):
            line = title[index]
            title_matches = re.findall(title_regex, line.lower())
            
            if bool(title_matches) and len(title_matches) > 0:
                for match in title_matches:
                    match = match.replace(' ', '_')
                    match = text_preprocessing.remove_accents(match)
                    features_dict[f'{match}_t'] += 1
                    vocabulary.add(match)
                    idfs['']
                
        content_matches = re.findall(content_regex, content.lower())
        
        for match in content_matches:
            match = match.replace(' ', '_')
            match = text_preprocessing.remove_accents(match)
            features_dict[f'{match}_c'] += 1
            vocabulary.add(match)
            
    return features_dict

In [ ]:
cities_data = defaultdict(list)
empty = 0

for city in ["pirapetinga", "coqueiral", "cristais", "olaria", "passa vinte", "arantina", "ijaci", "sao bento abade"]:
    city_pdfs = [info for info in pdfs if info["city"] == city]
    ata_count, homolog_count, edital_count, others_count = (0,)*4
    
    pdf_i = 0
    print('-'*80)
    print(f'## {city} ##')
    
    while pdf_i < len(city_pdfs):
        try:
            pdf = city_pdfs[pdf_i]
            pdf_file = pdfplumber.open(pdf['path'])
            pdf_id = pdf["id"].split(".")[0]
        
            pages = get_pages(pdf_file)
            content = get_first_tokens(pdf_file)
            title = title_extraction_breaklines(pdf_file)
            
            if (bool(title) and len(title)) == 0 or not (content):
                empty += 1
                print("ERRO! DOCUMENTO ESCANEADO")
            else:
                tokens_matches_dict = get_content_matches(pdf_id, title, content)
                tokens_matches_dict.update({'num_pages': len(pages)})
                cities_data[city].append(tokens_matches_dict)
                pdf_file.close()
        except Exception as e:
            print("*-*-*", e)
        pdf_i += 1

### old

In [45]:
vocabulary = set()
idfs = defaultdict(lambda: defaultdict(int))

default_features_dict = {'contrato_administrativo_t': 0, 'contrato_administrativo_c': 0, 'retificacao_t': 0, 'retificacao_c': 0, 
            'aviso_de_t': 0, 'aviso_de_c': 0, 'aditamento_t': 0, 'aditamento_c': 0, 'extrato_t': 0, 'extrato_c': 0, 
            'ordem_de_servico_t': 0, 'ordem_de_servico_c': 0, 'sessao_publica_t': 0, 'sessao_publica_c': 0, 
            'adjudicacao_t': 0, 'adjudicacao_c': 0, 'ata_t': 0, 'ata_c': 0, 'homologacao_t': 0, 'homologacao_c': 0, 
            'convite_t': 0, 'convite_c': 0, 'resposta_t': 0, 'resposta_c': 0, 'edital_t': 0, 'edital_c': 0, 
            'cronograma_t': 0, 'cronograma_c': 0, 'diario_oficial_t': 0, 'diario_oficial_c': 0}

def get_content_matches(pdf_id, title, content):
    features_dict = {'doc_id': pdf_id, 'title': title, 'city': city} 
    features_dict.update(copy.copy(default_features_dict))
    
    for word_dict in keywords:
        word = word_dict["word"]
        title_regex = word_dict["title_regex"]
        content_regex = word_dict["title_regex"]
        doc_class = word_dict["class"].lower()
        title_matches = []
        
        for index in range(len(title)):
            line = title[index]
            title_matches = re.findall(title_regex, line.lower())
            
            if bool(title_matches) and len(title_matches) > 0:
                for match in title_matches:
                    match = match.replace(' ', '_')
                    match = text_preprocessing.remove_accents(match)
                    features_dict[f'{match}_t'] += 1
                    vocabulary.add(match)
                    idfs['']
                
        content_matches = re.findall(content_regex, content.lower())
        
        for match in content_matches:
            match = match.replace(' ', '_')
            match = text_preprocessing.remove_accents(match)
            features_dict[f'{match}_c'] += 1
            vocabulary.add(match)
            
    return features_dict

In [46]:
cities = set()
for folder_path in folder_paths:
    for _, filepath in zip(range(limit), glob.iglob(folder_path)):
        try:
            pdf_id = filepath.split("/")[-1]
            pdf_city = (
                filepath.split("/")[2]
                .split("licitacoes")[-1]
                .replace("-", "", 1)
                .replace("-", " ")
            )
            cities.add(pdf_city)
            pdfs.append({"id": pdf_id, "city": pdf_city, "path": filepath})
        except Exception as e:
            print("**", e)

In [49]:
len(city_pdfs)

800

In [47]:
cities_data = defaultdict(list)
empty = 0

for city in ["pirapetinga", "coqueiral", "cristais", "olaria", "passa vinte", "arantina", "ijaci", "sao bento abade"]:
    city_pdfs = [info for info in pdfs if info["city"] == city]
    ata_count, homolog_count, edital_count, others_count = (0,)*4
    
    pdf_i = 0
    print('-'*80)
    print(f'## {city} ##')
    
    while pdf_i < len(city_pdfs):
        try:
            pdf = city_pdfs[pdf_i]
            pdf_file = pdfplumber.open(pdf['path'])
            pdf_id = pdf["id"].split(".")[0]
        
            pages = get_pages(pdf_file)
            content = get_first_tokens(pdf_file)
            title = title_extraction_breaklines(pdf_file)
            
            if (bool(title) and len(title)) == 0 or not (content):
                empty += 1
                print("ERRO! DOCUMENTO ESCANEADO")
            else:
                tokens_matches_dict = get_content_matches(pdf_id, title, content)
                tokens_matches_dict.update({'num_pages': len(pages)})
                cities_data[city].append(tokens_matches_dict)
                pdf_file.close()
        except Exception as e:
            print("*-*-*", e)
        pdf_i += 1

--------------------------------------------------------------------------------
## pirapetinga ##
--------------------------------------------------------------------------------
## coqueiral ##
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
*-*-* 'NoneType' object is not subscriptable
--------------------------------------------------------------------------------
## 

KeyboardInterrupt: 

In [ ]:
l = []
for a in vocabulary:
    l.append(f"{a}_t")
    l.append(f"{a}_c")

l

In [30]:
for city, files in cities_data.items():
    city = city.replace(' ', '_')
    df = pd.DataFrame(files)
    df.set_index('doc_id')
    df.to_csv(f'../data/keyword-classifier-data/{city}_input_data.csv')

In [11]:
pd.DataFrame(cities_data['coqueiral'])

,contrato_administrativo_t,contrato_administrativo_c,retificacao_t,retificacao_c,aviso_de_t,aviso_de_c,aditamento_t,aditamento_c,extrato_t,extrato_c,...,edital_t,edital_c,cronograma_t,cronograma_c,diario_oficial_t,diario_oficial_c,doc_id,title,city,num_pages
0,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
1,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
2,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
3,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
4,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
388,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
389,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30
390,163,371,20,28,132,152,58,115,52,275,...,660,3479,1,10,157,189,fe9dc48f8368aae9c5be0e995dd4214d,"[ , prefeitura municipal de são bento abade mg...",sao bento abade,30


In [ ]:
data_list = []

for city, files in cities_data.items():
    for file in files:
        data_list.append(
            {'doc_id': file['doc_id'], 'city': city, 'pages': len(file['num_pages']),
             'contrato administrativo_t': file['doc_id'], 'contrato administrativo_c', 'retificação_t', 'retificação_c', 
        'aviso de_t', 'aviso de_c', 'aditamento_t', 'aditamento_c', 'extrato_t', 'extrato_c',
        'ordem de serviço_t', 'ordem de serviço_c', 'sessão pública_t', 'sessão pública_c', 
        'adjudicação_t', 'adjudicação_c', 'ata_t', 'ata_c', 'homologação_t', 'homologação_c', 
        'convite_t', 'convite_c', 'resposta_t', 'resposta_c', 'edital_t', 'edital_c', 
        'cronograma_t', 'cronograma_c', 'diário oficial_t', 'diário oficial_c'})

In [ ]:
cities_data_2 = cities_data
for city, files in cities_data.items():
    for file in files:
        